In [1]:
using JuMP;
using Clp;
using Distributions;

# srand(123456)
N_seq = [100 1000 10000];

for i in 1:length(N_seq) 
    # parameters
    N = N_seq[i];
    
    distrb1 = Uniform(-0.8, 0.8);
    distrb2 = Normal(0, 12);
    distrb3 = Exponential(2.5);
    distrb4 = Normal(0, 9);
    
    xi1 = rand(distrb1, N);
    xi2 = rand(distrb2, N);
    xi3 = rand(distrb3, N);
    xi4 = rand(distrb4, N);
    
    Master=Model(solver=ClpSolver());
    
    @variable(Master, x[1:2]>=0);
    UBtheta = 1000000;
    @variable(Master, 0<=theta[1:N]<=UBtheta);
    
    @constraint(Master, x[1]+x[2] <= 100);
    @objective(Master, Min, 2*x[1]+3*x[2]+sum(theta[k]*1.0/N for k=1:N));
    
    # check if there need feasible cut
    status = solve(Master);
    xval = getvalue(x);
    
    # give arbitrary xval initially
    xval = [0 0];
    
    # Construct 2nd stage problem
    Subprob = Model(solver=ClpSolver());
    @variable(Subprob, y[1:2]>=0);
    @objective(Subprob, Min, 7*y[1]+12*y[2]);
    # change the RHS later
    @constraint(Subprob, constr1, y[1]>=0);
    @constraint(Subprob, constr2, y[2]>=0);
    
    LB=0;
    UB=1e10;
    iter=0;
    iter1=0;
        
    while (UB-LB)*1.0/UB > 1e-5
        iter1 = iter1 + 1;
        # First solve the master problem, and get x value
        status = solve(Master);
        xval = getvalue(x);
        thetaval = getvalue(theta);
        # println("theta = ", thetaval);
        LB = getobjectivevalue(Master);
        
        duals = zeros(2);
        tempUB = 0;
        tempUB = tempUB + 2*xval[1] + 3*xval[2];
        x1coef = zeros(N);
        x2coef = zeros(N);
        constant = zeros(N);
    
        for k=1:N
            JuMP.setRHS(constr1, 180+xi2[k]-((2+xi1[k])*xval[1]+6*xval[2]));
            JuMP.setRHS(constr2, 162+xi4[k]-(3*xval[1]+(3.4-xi3[k])*xval[2]));
            
            status = solve(Subprob);
            tempUB = tempUB + getobjectivevalue(Subprob)*1.0/N;
            duals[1] = getdual(constr1);
            duals[2] = getdual(constr2);
            
            x1coef[k] = (2+xi1[k])*duals[1]+3*duals[2];
            x2coef[k] = 6*duals[1]+(3.4-xi3[k])*duals[2];
            constant[k] = (180+xi2[k])*duals[1] + (162+xi4[k])*duals[2];
        end
        
        if tempUB < UB
            UB = tempUB;
        end
        
        if (UB-LB)*1.0/UB < 1e-5
            break;
        else
            for k=1:N
                # check if the cut is violated by current (xval,thetaval) by at least 1e-5
                if thetaval[k]<constant[k]-x1coef[k]*xval[1]-x2coef[k]*xval[2]-1e-5
                    iter = iter + 1;
                    @constraint(Master,theta[k]>=constant[k]-x1coef[k]*x[1]-x2coef[k]*x[2]);
                    # comment out to show (tons of output)
                    # @printf "cut[%d]: scenario = %d, cutcoef1 = %f, cutcoef2 = %f, cutrhs = %f\n" iter k x1coef[k] x2coef[k] constant[k];
                end
            end
        end
    end
    println("############################# N = ", N, " #############################")
    println("LB = ", LB);
    println("UB = ", UB);
    println("xval1 = ", xval[1]);
    println("xval2 = ", xval[2]);
end

############################# N = 100 #############################
LB = 202.09445509945527
UB = 202.09445509945525
xval1 = 77.87174812001517
xval2 = 12.098962548578013


############################# N = 1000 #############################
LB = 208.29449850213595
UB = 208.294498502136
xval1 = 77.59491595283725
xval2 = 12.470244641280358
############################# N = 10000 #############################
LB = 210.46752341584656
UB = 210.46752341584656
xval1 = 78.44089883917228
xval2 = 11.864417102374565
